In [139]:
from pathlib import Path 
import requests
import pandas as pd
import numpy as np
from io import StringIO
from simple_salesforce import Salesforce
import shutil
import datetime
from datetime import date


sf = Salesforce(
    username="jmartinez@careassistance.com",
    password="Jcmm0104",
    security_token="EwVPSL02qXM8XQ2pkUdA13aW",
)

Descarga de pacientes y empresas

In [122]:
# Recuperar empresas ID de salesforce desde Account
results = sf.query_all(
    """
   select Id, Name, IdentificationId__pc, Identification_Type__c,Holding__c ,Pa_s_del_Paciente__c,Campa_a__c,AccountNumber,Id_Empresa__c,poliza__c,Empleador__c  from account where ispersonaccount = false 
    """
)
Empresas_SF = pd.DataFrame(results["records"])
Empresas_SF = Empresas_SF[
    ["Id", "Name", "IdentificationId__pc", "Identification_Type__c", "Holding__c", "Pa_s_del_Paciente__c",'Campa_a__c','AccountNumber','Id_Empresa__c','Empleador__c','poliza__c']
]
Empresas_SF.shape

(7886, 11)

In [123]:
# recuperar todos los pacientes ID desdde aAccount
results = sf.query_all(
    """
  select Id, IdentificationId__pc,Name,poliza__c, Identification_Type__c, IsPersonAccount, Ocupaci_n__c, Pa_s_del_Paciente__c, PersonTitle ,PersonDepartment  from Account where ispersonaccount = True
    """
)
Pacientes_SF = pd.DataFrame(results["records"])

Pacientes_SF = Pacientes_SF[["Id", "IdentificationId__pc", "Name",'poliza__c','Identification_Type__c','IsPersonAccount','Ocupaci_n__c','Pa_s_del_Paciente__c','PersonTitle','PersonDepartment']]
Pacientes_SF.shape

(1534724, 10)

In [124]:
# recuperar todos los pacientes ID desde Contact
results = sf.query_all(
    """
   select Id,Name,AccountId  from Contact
    """
)
Pacientesc_SF = pd.DataFrame(results["records"])

Pacientesc_SF = Pacientesc_SF[["Id", "AccountId", "Name"]]
Pacientesc_SF.shape

(1534724, 3)

In [125]:
#Relacion AccountContactRelacion se descarga la relacion paciente Empresa en account contact no es la q se ve
results = sf.query_all(
    """
   select Id,AccountId,ContactId,NameAccount__c, RutContact__c from AccountContactRelation
    """
)
Relacion_Pacientes_Empresas = pd.DataFrame(results["records"])

Relacion_Pacientes_Empresas = Relacion_Pacientes_Empresas[
    ["Id", "AccountId", "ContactId", "NameAccount__c", "RutContact__c"]
]
Relacion_Pacientes_Empresas.shape

(1222508, 5)

In [126]:
#union de pacientes
Union_account_contact_relation = pd.merge(
    left=Relacion_Pacientes_Empresas,
    right=Pacientesc_SF,
    how="left",
    left_on="ContactId",
    right_on="Id",
)
Union_account_contact_relation.head()

,Id_x,AccountId_x,ContactId,NameAccount__c,RutContact__c,Id_y,AccountId_y,Name
0,07k8c00000RGWPNAA5,0018c00002axQa5AAE,0038c000036jLDrAAM,Metlife Holding Metlife Finning,174365172,0038c000036jLDrAAM,0018c00002cVCmPAAW,Correa Aguilar Rene Alberto
1,07k8c00000RGWPAAA5,0018c00002axQdlAAE,0038c000036jelbAAA,Metlife Holding Metlife Skberge,182204846,0038c000036jelbAAA,0018c00002cVWElAAO,Maldonado Velasquez Nicolas
2,07k8c00000RGWPBAA5,0018c00002axQdlAAE,0038c000035CWLYAA4,Metlife Holding Metlife Skberge,18396206K,0038c000035CWLYAA4,0018c00002amMfoAAE,Zaror Yunis Daniel Esteban
3,07k8c00000RGWPCAA5,0018c00002axQdlAAE,0038c000035CaxiAAC,Metlife Holding Metlife Skberge,184262452,0038c000035CaxiAAC,0018c00002amRHeAAM,Bustamante Morales Flor Del Carmen
4,07k8c00000RGWPDAA5,0018c00002axQdlAAE,0038c000036jelcAAA,Metlife Holding Metlife Skberge,184429772,0038c000036jelcAAA,0018c00002cVWEmAAO,Malleo Abarca Rigoberto Andres


In [127]:
#union de empresas
Union_account_contact_relation = pd.merge(
    left=Union_account_contact_relation,
    right=Empresas_SF,
    how="left",
    left_on="AccountId_x",
    right_on="Id",
)
Union_account_contact_relation.head()

,Id_x,AccountId_x,ContactId,NameAccount__c,RutContact__c,Id_y,AccountId_y,Name_x,Id,Name_y,IdentificationId__pc,Identification_Type__c,Holding__c,Pa_s_del_Paciente__c,Campa_a__c,AccountNumber,Id_Empresa__c,Empleador__c,poliza__c
0,07k8c00000RGWPNAA5,0018c00002axQa5AAE,0038c000036jLDrAAM,Metlife Holding Metlife Finning,174365172,0038c000036jLDrAAM,0018c00002cVCmPAAW,Correa Aguilar Rene Alberto,0018c00002axQa5AAE,Metlife Holding Metlife Finning,None,None,MetLife,a1U8c000005dAtnEAE,MetLife,None,EMP5951,None,None
1,07k8c00000RGWPAAA5,0018c00002axQdlAAE,0038c000036jelbAAA,Metlife Holding Metlife Skberge,182204846,0038c000036jelbAAA,0018c00002cVWElAAO,Maldonado Velasquez Nicolas,0018c00002axQdlAAE,Metlife Holding Metlife Skberge,None,None,MetLife,a1U8c000005dAtnEAE,MetLife,None,EMP5641,None,None
2,07k8c00000RGWPBAA5,0018c00002axQdlAAE,0038c000035CWLYAA4,Metlife Holding Metlife Skberge,18396206K,0038c000035CWLYAA4,0018c00002amMfoAAE,Zaror Yunis Daniel Esteban,0018c00002axQdlAAE,Metlife Holding Metlife Skberge,None,None,MetLife,a1U8c000005dAtnEAE,MetLife,None,EMP5641,None,None
3,07k8c00000RGWPCAA5,0018c00002axQdlAAE,0038c000035CaxiAAC,Metlife Holding Metlife Skberge,184262452,0038c000035CaxiAAC,0018c00002amRHeAAM,Bustamante Morales Flor Del Carmen,0018c00002axQdlAAE,Metlife Holding Metlife Skberge,None,None,MetLife,a1U8c000005dAtnEAE,MetLife,None,EMP5641,None,None
4,07k8c00000RGWPDAA5,0018c00002axQdlAAE,0038c000036jelcAAA,Metlife Holding Metlife Skberge,184429772,0038c000036jelcAAA,0018c00002cVWEmAAO,Malleo Abarca Rigoberto Andres,0018c00002axQdlAAE,Metlife Holding Metlife Skberge,None,None,MetLife,a1U8c000005dAtnEAE,MetLife,None,EMP5641,None,None


In [128]:
#union de pacientes con sus datos
Union_account_contact_relation = pd.merge(
    left=Union_account_contact_relation,
    right=Pacientes_SF,
    how="left",
    left_on="RutContact__c",
    right_on="IdentificationId__pc",
)
Union_account_contact_relation.head()

C:\Users\jmartinez\AppData\Local\Temp\ipykernel_19232\3766479298.py:2: FutureWarning: Passing 'suffixes' which cause duplicate columns {'Id_x'} in the result is deprecated and will raise a MergeError in a future version.
  Union_account_contact_relation = pd.merge(


,Id_x,AccountId_x,ContactId,NameAccount__c,RutContact__c,Id_y,AccountId_y,Name_x,Id_x,Name_y,...,Id_y,IdentificationId__pc_y,Name,poliza__c_y,Identification_Type__c_y,IsPersonAccount,Ocupaci_n__c,Pa_s_del_Paciente__c_y,PersonTitle,PersonDepartment
0,07k8c00000RGWPNAA5,0018c00002axQa5AAE,0038c000036jLDrAAM,Metlife Holding Metlife Finning,174365172,0038c000036jLDrAAM,0018c00002cVCmPAAW,Correa Aguilar Rene Alberto,0018c00002axQa5AAE,Metlife Holding Metlife Finning,...,0018c00002cVCmPAAW,174365172,Correa Aguilar Rene Alberto,None,RUT,True,None,a1U8c000005dAtnEAE,None,None
1,07k8c00000RGWPAAA5,0018c00002axQdlAAE,0038c000036jelbAAA,Metlife Holding Metlife Skberge,182204846,0038c000036jelbAAA,0018c00002cVWElAAO,Maldonado Velasquez Nicolas,0018c00002axQdlAAE,Metlife Holding Metlife Skberge,...,0018c00002cVWElAAO,182204846,Maldonado Velasquez Nicolas,None,RUT,True,None,a1U8c000005dAtnEAE,None,None
2,07k8c00000RGWPBAA5,0018c00002axQdlAAE,0038c000035CWLYAA4,Metlife Holding Metlife Skberge,18396206K,0038c000035CWLYAA4,0018c00002amMfoAAE,Zaror Yunis Daniel Esteban,0018c00002axQdlAAE,Metlife Holding Metlife Skberge,...,0018c00002amMfoAAE,18396206K,Zaror Yunis Daniel Esteban,None,RUT,True,None,a1U8c000005dAtnEAE,None,None
3,07k8c00000RGWPCAA5,0018c00002axQdlAAE,0038c000035CaxiAAC,Metlife Holding Metlife Skberge,184262452,0038c000035CaxiAAC,0018c00002amRHeAAM,Bustamante Morales Flor Del Carmen,0018c00002axQdlAAE,Metlife Holding Metlife Skberge,...,0018c00002amRHeAAM,184262452,Bustamante Morales Flor Del Carmen,None,RUT,True,None,a1U8c000005dAtnEAE,None,None
4,07k8c00000RGWPDAA5,0018c00002axQdlAAE,0038c000036jelcAAA,Metlife Holding Metlife Skberge,184429772,0038c000036jelcAAA,0018c00002cVWEmAAO,Malleo Abarca Rigoberto Andres,0018c00002axQdlAAE,Metlife Holding Metlife Skberge,...,0018c00002cVWEmAAO,184429772,Malleo Abarca Rigoberto Andres,None,RUT,True,None,a1U8c000005dAtnEAE,None,None


In [129]:
#buscar empresas desde Programas_por_Empresas__c
results = sf.query_all(
    """
   select Id,Empresa__c, Empresa_Padre__c,Nombre_Empresa__c, Name  from Programas_por_Empresas__c 
    """
)
Programas_por_empresa_SF = pd.DataFrame(results["records"])

Programas_por_empresa_SF = Programas_por_empresa_SF[
    ["Id", "Empresa__c", "Empresa_Padre__c", "Nombre_Empresa__c", "Name"]
]
Programas_por_empresa_SF.shape

(7892, 5)

In [130]:
#buscar empresas desde Paciente_Programas__c
results = sf.query_all(
    """
    select Id,Paciente__c, Programas_por_Empresas__c,padre__c from Paciente_Programas__c  
    """
)
Paciente_Programas_SF = pd.DataFrame(results["records"])

Paciente_Programas_SF = Paciente_Programas_SF[
    ["Id", "Paciente__c", "Programas_por_Empresas__c", "padre__c"]
]
Paciente_Programas_SF.shape

(1222351, 4)

In [131]:
#unir empresas y todas sus caracteristicas


empresas_unidas = pd.merge(
    left=Paciente_Programas_SF,
    right=Programas_por_empresa_SF,
    how="left",
    left_on="Programas_por_Empresas__c",
    right_on="Id",
)
empresas_unidas.shape

(1222351, 9)

In [132]:
empresas_unidas.head()
apprix_1 = empresas_unidas.iloc[:611178,:]
apprix_2 = empresas_unidas.iloc[611178:,:]
apprix_1.to_excel(
    "C:/Users/jmartinez/Documents/empresasA.xlsx", index=False
)
apprix_2.to_excel(
    "C:/Users/jmartinez/Documents/empresasB.xlsx", index=False
)

Descarga desde Citas de Servicios

In [208]:
# Citas de Servicio
results = sf.query_all(
    """
    SELECT Id, clinica__c,AccountId,CreatedDate,RUT__c,WorkTypeId,Work_TypeGroup__c,Status,Interacci_n__c,AppointmentNumber,ParentRecordId,Tipo_de_la_cita__c from ServiceAppointment
    """
)
Citas_de_servicio = pd.DataFrame(results["records"])
Citas_de_servicio = Citas_de_servicio[
    [
        "AppointmentNumber",
        
        "RUT__c",
        'Clinica__c',
        "CreatedDate",
        
        "Tipo_de_la_cita__c",
        "AccountId",
        "WorkTypeId" 
    ]
]
Citas_de_servicio.shape


(6930, 7)

In [209]:
# Citas de Servicio Orientacion
results = sf.query_all(
    """
    SELECT Id, AccountId,CreatedDate,RUT__c,WorkTypeId,Work_TypeGroup__c,Status,Interacci_n__c,AppointmentNumber,ParentRecordId,Tipo_de_la_cita__c from ServiceAppointment
    """
)
Citas_de_servicio2 = pd.DataFrame(results["records"])
Citas_de_servicio2['Clinica__c'] = 'Orientación Médica'
Citas_de_servicio2 = Citas_de_servicio2[
    [
        "AppointmentNumber",
        "RUT__c",
        'Clinica__c',
        "CreatedDate",
        
        "Tipo_de_la_cita__c",
        "AccountId",
        "WorkTypeId" 
    ]
]
Citas_de_servicio2.shape


(6930, 7)

In [210]:
#unir dos citas
Gran_Citas_deservicio = pd.concat([Citas_de_servicio, Citas_de_servicio2], axis=0)

In [211]:
Gran_Citas_deservicio.shape

(13860, 7)

In [212]:
Gran_Citas_deservicio.head()

,AppointmentNumber,RUT__c,Clinica__c,CreatedDate,Tipo_de_la_cita__c,AccountId,WorkTypeId
0,SA-3124,183552252,Clínica de Nutrición,2022-12-02T16:23:05.000+0000,Phone,0018c00002azpJ4AAI,08q8c000000VW1yAAG
1,SA-4987,224488556,Clínica de Nutrición,2022-12-21T21:28:49.000+0000,Phone,0018c00002amLbpAAE,08q8c000000VW1yAAG
2,SA-3905,185384799,Clínica de Deporte,2022-12-12T15:34:28.000+0000,Phone,0018c00002ay6KFAAY,08q8c000000VW24AAG
3,SA-3871,187821444,Clínica de Psicología,2022-12-12T12:21:02.000+0000,Phone,0018c00002bI5vJAAS,08q8c000000VVsBAAW
4,SA-3898,167466923,Clínica de Nutrición,2022-12-12T15:04:05.000+0000,Phone,0018c00002ao4fCAAQ,08q8c000000VVs1AAG


In [213]:
#Clinica_deporte
results = sf.query_all(
    """
   SELECT id,Name,Rut__c,Nombre_Empresa__c,CreatedDate,Canal__c, Empresa__c, Account__c,RecordTypeId from Ficha_Cl_nica_Deporte__c
     """
)
ficha_deporte = pd.DataFrame(results["records"])

ficha_deporte = ficha_deporte[
    [
        "Name",
        "Rut__c",
        "Nombre_Empresa__c",
        "CreatedDate",
        "Canal__c",
        "Empresa__c",
        "Account__c",
        'RecordTypeId'
    ]
]
ficha_deporte['clinica'] = 'Deporte'
ficha_deporte = ficha_deporte.rename(
    columns={
        "Name": "Codigo Ficha",
        "Rut__c": "Rut",
        "Nombre_Empresa__c": "Empleador",
        "CreatedDate": "Fecha",
        "Canal__c": "Canal",
        "Empresa__c": "Cod. Empresa Emp_pro",
        "Account__c": "Id Paciente",
        "RecordTypeId": "Tipo Ficha",
        'clinica':'clínica'
    }
)
ficha_deporte.shape


(695, 9)

In [214]:
# Clinica Nutricion
results = sf.query_all(
    """
   select  Name, RUT__c, Nombre_Empresa__c, CreatedDate, Canal__c, Empresa__c, Account__c, RecordTypeId from Ficha_Clinica_Nutrici_n__c 
     """
)
ficha_nutricion = pd.DataFrame(results["records"])

ficha_nutricion = ficha_nutricion[
    [
        "Name",
        "RUT__c",
        "Nombre_Empresa__c",
        "CreatedDate",
        "Canal__c",
        "Empresa__c",
        "Account__c",
        "RecordTypeId",
    ]
]
ficha_nutricion["clinica"] = "Nutrición"

ficha_nutricion = ficha_nutricion.rename(
    columns={
        "Name": "Codigo Ficha",
        "RUT__c": "Rut",
        "Nombre_Empresa__c": "Empleador",
        "CreatedDate": "Fecha",
        "Canal__c": "Canal",
        "Empresa__c": "Cod. Empresa Emp_pro",
        "Account__c": "Id Paciente",
        "RecordTypeId": "Tipo Ficha",
        'clinica':'clínica'
    }
)


ficha_nutricion.shape

(1624, 9)

In [215]:
#Clinica AppSex
results = sf.query_all(
    """
   select  Name, Rut__c, Nombre_Empresa__c, CreatedDate, Canal__c, Empresa__c, Account__c, RecordTypeId from Ficha_Cl_nica_ApsSex__c 
     """
)
ficha_Apssex = pd.DataFrame(results["records"])

ficha_Apssex = ficha_Apssex[
    [
        "Name",
        "Rut__c",
        "Nombre_Empresa__c",
        "CreatedDate",
        "Canal__c",
        "Empresa__c",
        "Account__c",
        'RecordTypeId'
    ]
]
ficha_Apssex['clinica'] = 'Asesoría Psicologíca en Salud Sexual'
ficha_Apssex = ficha_Apssex.rename(
    columns={
        "Name": "Codigo Ficha",
        "Rut__c": "Rut",
        "Nombre_Empresa__c": "Empleador",
        "CreatedDate": "Fecha",
        "Canal__c": "Canal",
        "Empresa__c": "Cod. Empresa Emp_pro",
        "Account__c": "Id Paciente",
        "RecordTypeId": "Tipo Ficha",
        'clinica':'clínica'
    }
)

ficha_Apssex.shape

(3, 9)

In [216]:
#Clinica Psicologia
results = sf.query_all(
    """
   select  Name, Rut__c, Nombre_Empresa__c, CreatedDate, Canal__c, Empresa__c, Account__c, RecordTypeId from Ficha_Cl_nica_Asistencia_Emocional__c 
     """
)
ficha_Psicologia = pd.DataFrame(results["records"])

ficha_Psicologia = ficha_Psicologia[
    [
        "Name",
        "Rut__c",
        "Nombre_Empresa__c",
        "CreatedDate",
        "Canal__c",
        "Empresa__c",
        "Account__c",
        'RecordTypeId'
    ]
]
ficha_Psicologia['clinica'] = 'Psicología'
ficha_Psicologia = ficha_Psicologia.rename(
    columns={
        "Name": "Codigo Ficha",
        "Rut__c": "Rut",
        "Nombre_Empresa__c": "Empleador",
        "CreatedDate": "Fecha",
        "Canal__c": "Canal",
        "Empresa__c": "Cod. Empresa Emp_pro",
        "Account__c": "Id Paciente",
        "RecordTypeId": "Tipo Ficha",
        'clinica':'clínica'
    }
)
ficha_Psicologia.shape

(1213, 9)

In [217]:
#Clinica Sueño
results = sf.query_all(
    """
   select  Name, RUT__c, Nombre_Empresa__c, CreatedDate, Canal__c, Empresa__c, Account__c, RecordTypeId from Ficha_Cl_nica_del_Sue_o__c 
     """
)
ficha_Sueno = pd.DataFrame(results["records"])
ficha_Sueno = ficha_Sueno[
    [
        "Name",
        "Rut__c",
        "Nombre_Empresa__c",
        "CreatedDate",
        "Canal__c",
        "Empresa__c",
        "Account__c",
        'RecordTypeId'
    ]
]
ficha_Sueno['clinica'] = 'Clínica Del Sueño'
ficha_Sueno = ficha_Sueno.rename(
    columns={
        "Name": "Codigo Ficha",
        "Rut__c": "Rut",
        "Nombre_Empresa__c": "Empleador",
        "CreatedDate": "Fecha",
        "Canal__c": "Canal",
        "Empresa__c": "Cod. Empresa Emp_pro",
        "Account__c": "Id Paciente",
        "RecordTypeId": "Tipo Ficha",
        'clinica':'clínica'
    }
)
ficha_Sueno.shape

(178, 9)

In [218]:
#Clinica PAciente Cronico
results = sf.query_all(
    """
   select  Name, RUT__c, Nombre_Empresa__c, CreatedDate, Canal__c, Empresa__c, Account__c, RecordTypeId from Ficha_Cl_nica_Seg_Paciente_Cr_nico__c  
     """
)
ficha_Cronico = pd.DataFrame(results["records"])
ficha_Cronico = ficha_Cronico[
    [
        "Name",
        "Rut__c",
        "Nombre_Empresa__c",
        "CreatedDate",
        "Canal__c",
        "Empresa__c",
        "Account__c",
        'RecordTypeId'
    ]
]
ficha_Cronico['clinica'] = 'Pacientes Crónicos'
ficha_Cronico = ficha_Cronico.rename(
    columns={
        "Name": "Codigo Ficha",
        "Rut__c": "Rut",
        "Nombre_Empresa__c": "Empleador",
        "CreatedDate": "Fecha",
        "Canal__c": "Canal",
        "Empresa__c": "Cod. Empresa Emp_pro",
        "Account__c": "Id Paciente",
        "RecordTypeId": "Tipo Ficha",
        'clinica':'clínica'
    }
)
ficha_Cronico.shape

(514, 9)

In [219]:
#Clinica Telemedicina
results = sf.query_all(
    """
   select  Name, RUT__c, Nombre_Empresa__c, CreatedDate, Canal__c, Empresa__c, Account__c, RecordTypeId from Ficha_Cl_nica_Telemedicina__c   
     """
)
ficha_Telemedicina = pd.DataFrame(results["records"])
ficha_Telemedicina = ficha_Telemedicina[
    [
        "Name",
        "Rut__c",
        "Nombre_Empresa__c",
        "CreatedDate",
        "Canal__c",
        "Empresa__c",
        "Account__c",
        'RecordTypeId'
    ]
]
ficha_Telemedicina['clinica'] = 'Telemedicina'
ficha_Telemedicina = ficha_Telemedicina.rename(
    columns={
        "Name": "Codigo Ficha",
        "Rut__c": "Rut",
        "Nombre_Empresa__c": "Empleador",
        "CreatedDate": "Fecha",
        "Canal__c": "Canal",
        "Empresa__c": "Cod. Empresa Emp_pro",
        "Account__c": "Id Paciente",
        "RecordTypeId": "Tipo Ficha",
        'clinica':'clínica'
    }
)
ficha_Telemedicina.shape

(2109, 9)

In [220]:
#Clinica Veterinaria
results = sf.query_all(
    """
   select  Name, RUT__c, Nombre_Empresa__c, CreatedDate, Canal__c, Empresa__c, Account__c, RecordTypeId from Ficha_Cl_nica_Veterinaria__c    
     """
)
ficha_Veterinaria = pd.DataFrame(results["records"])
ficha_Veterinaria = ficha_Veterinaria[
    [
        "Name",
        "Rut__c",
        "Nombre_Empresa__c",
        "CreatedDate",
        "Canal__c",
        "Empresa__c",
        "Account__c",
        'RecordTypeId'
    ]
]
ficha_Veterinaria['clinica'] = 'Veterinaria'
ficha_Veterinaria = ficha_Veterinaria.rename(
    columns={
        "Name": "Codigo Ficha",
        "Rut__c": "Rut",
        "Nombre_Empresa__c": "Empleador",
        "CreatedDate": "Fecha",
        "Canal__c": "Canal",
        "Empresa__c": "Cod. Empresa Emp_pro",
        "Account__c": "Id Paciente",
        "RecordTypeId": "Tipo Ficha",
        'clinica':'clínica'
    }
)
ficha_Veterinaria.shape

(6, 9)

In [221]:
#Clinica Orientacion
results = sf.query_all(
    """
   select  Name, RUT__c, Nombre_Empresa__c, CreatedDate, Canal__c, Empresa__c, Account__c from Ficha_Orientaci_n_Medica__c     
     """
)
ficha_Orientacion = pd.DataFrame(results["records"])
ficha_Orientacion['RecordTypeId'] = 'Ficha de Derivacion'
ficha_Orientacion = ficha_Orientacion[
    [
        "Name",
        "Rut__c",
        "Nombre_Empresa__c",
        "CreatedDate",
        "Canal__c",
        "Empresa__c",
        "Account__c",
        'RecordTypeId'
    ]
]
ficha_Orientacion['clinica'] = 'Orientación Médica'
ficha_Orientacion = ficha_Orientacion.rename(
    columns={
        "Name": "Codigo Ficha",
        "Rut__c": "Rut",
        "Nombre_Empresa__c": "Empleador",
        "CreatedDate": "Fecha",
        "Canal__c": "Canal",
        "Empresa__c": "Cod. Empresa Emp_pro",
        "Account__c": "Id Paciente",
        "RecordTypeId": "Tipo Ficha",
        'clinica':'clínica'
    }
)
ficha_Orientacion.shape

(5, 9)

In [222]:
#Clinica Front
results = sf.query_all(
    """
   select  Name, RUT__c, Nombre_Empresa__c, CreatedDate, Canal__c, Empresa__c, Account__c,RecordTypeId from Ficha_Front__c     
     """
)
ficha_Front = pd.DataFrame(results["records"])

ficha_Front = ficha_Front[
    [
        "Name",
        "Rut__c",
        "Nombre_Empresa__c",
        "CreatedDate",
        "Canal__c",
        "Empresa__c",
        "Account__c",
        'RecordTypeId'
    ]
]
ficha_Front['clinica'] = 'Orientación Médica'
ficha_Front = ficha_Front.rename(
    columns={
        "Name": "Codigo Ficha",
        "Rut__c": "Rut",
        "Nombre_Empresa__c": "Empleador",
        "CreatedDate": "Fecha",
        "Canal__c": "Canal",
        "Empresa__c": "Cod. Empresa Emp_pro",
        "Account__c": "Id Paciente",
        "RecordTypeId": "Tipo Ficha",
        'clinica':'clínica'
    }
)
ficha_Front.shape

(35, 9)

In [223]:
#Clinica Ingreso deportivo
results = sf.query_all(
    """
   select  Name, RUT__c, Nombre_Empresa__c, CreatedDate, Canal__c, Empresa__c, Account__c,RecordTypeId from Ficha_Cl_nica_Ingreso_Deportivo__c    
     """
)
ficha_Ingreso_dep = pd.DataFrame(results["records"])

ficha_Ingreso_dep = ficha_Ingreso_dep[
    [
        "Name",
        "Rut__c",
        "Nombre_Empresa__c",
        "CreatedDate",
        "Canal__c",
        "Empresa__c",
        "Account__c",
        'RecordTypeId'
    ]
]
ficha_Ingreso_dep['clinica'] = 'Deporte'
ficha_Ingreso_dep = ficha_Ingreso_dep.rename(
    columns={
        "Name": "Codigo Ficha",
        "Rut__c": "Rut",
        "Nombre_Empresa__c": "Empleador",
        "CreatedDate": "Fecha",
        "Canal__c": "Canal",
        "Empresa__c": "Cod. Empresa Emp_pro",
        "Account__c": "Id Paciente",
        "RecordTypeId": "Tipo Ficha",
        'clinica':'clínica'
    }
)
ficha_Ingreso_dep.shape

(187, 9)

In [237]:
Clinicas = pd.concat([ficha_Front, ficha_Ingreso_dep,ficha_Orientacion,ficha_Veterinaria,ficha_Telemedicina,ficha_Cronico,ficha_Sueno,ficha_Psicologia,ficha_Apssex,ficha_nutricion,ficha_deporte], axis=0)
Clinicas.shape

(6569, 9)

In [232]:
Clinicas.head()

,Codigo Ficha,Rut,Empleador,Fecha,Canal,Cod. Empresa Emp_pro,Id Paciente,Tipo Ficha,clínica
0,FFF-0035,181523778,Farmacias Ahumada S.A.,2023-01-24T18:31:33.000+0000,Entrante - WhatsApp,a1Y8c00000C6VINEA3,0018c00002amVPcAAM,0128c000002CYubAAG,Orientación Médica
1,FFF-0037,184478412,Asociación Chilena De Seguridad,2023-01-24T19:19:01.000+0000,Saliente - Llamada,a1Y8c00000Ckvd5EAB,0018c00002ay08PAAQ,0128c000002CYubAAG,Orientación Médica
2,FFF-0038,150356237,Asociación Chilena De Seguridad,2023-01-24T19:21:28.000+0000,Saliente - Llamada,a1Y8c00000CkvAuEAJ,0018c00002axzuCAAQ,0128c000002CYubAAG,Orientación Médica
3,FFF-0036,154370366,Asociación Chilena De Seguridad,2023-01-24T18:41:51.000+0000,Saliente - Llamada,a1Y8c00000CkvYPEAZ,0018c00002ay05tAAA,0128c000002CYubAAG,Orientación Médica
4,FFF-0030,221476611,Metlife Holding Metlife Metlife,2023-01-24T12:18:02.000+0000,Entrante - Chat Online,a1Y8c00000CktIBEAZ,0018c00002axYJ7AAM,0128c000002CYubAAG,Orientación Médica


In [226]:
Gran_Citas_deservicio["Mes"] = pd.to_datetime(
    Gran_Citas_deservicio["CreatedDate"]
).dt.month

Gran_Citas_deservicio["Año"] = pd.to_datetime(
    Gran_Citas_deservicio["CreatedDate"]
).dt.year
Gran_Citas_deservicio["Dia"] = pd.to_datetime(
    Gran_Citas_deservicio["CreatedDate"]
).dt.day

# x = date(Citas_de_servicio['Dia'], Citas_de_servicio['Mes'], Citas_de_servicio['Año'] )
Gran_Citas_deservicio["Fecha"] = pd.to_datetime(
    Gran_Citas_deservicio["CreatedDate"], dayfirst=True, yearfirst=False, exact=True
)
Gran_Citas_deservicio["Fecha3"] = Gran_Citas_deservicio["Fecha"].dt.strftime("%d-%m-%Y")

clinicas_ = {
    "Clínica de Nutrición": "Nutrición",
    "Clínica de Deporte": "Deporte",
    "Clínica de Psicología": "Psicología",
    "Clínica Ingreso Deportivo Seguro": "Deporte",
    "Clínica del Sueño": "Clínica Del Sueño",
    "Clínica Seguimiento Pacientes Crónicos": "Pacientes Crónicos",
    'Orientación Médica':'Orientación Médica' ,
    'Asesoría Deportiva':'Deporte',
    'Asistencia Emocional':"Psicología",
    'Asistencia Nutricional':'Nutrición',
    'Clínica APSSex':'Asesoría Psicologíca en Salud Sexual',
    'Clínica Del Sueño':'Clínica Del Sueño',
    'Clínica Veterinaria':'Veterinaria',
    

    
}
Gran_Citas_deservicio.replace({"Clinica__c": clinicas_}, inplace=True)


Gran_Citas_deservicio.head()



,AppointmentNumber,RUT__c,Clinica__c,CreatedDate,Tipo_de_la_cita__c,AccountId,WorkTypeId,Mes,Año,Dia,Fecha,Fecha3
0,SA-3124,183552252,Nutrición,2022-12-02T16:23:05.000+0000,Phone,0018c00002azpJ4AAI,08q8c000000VW1yAAG,12,2022,2,2022-12-02 16:23:05+00:00,02-12-2022
1,SA-4987,224488556,Nutrición,2022-12-21T21:28:49.000+0000,Phone,0018c00002amLbpAAE,08q8c000000VW1yAAG,12,2022,21,2022-12-21 21:28:49+00:00,21-12-2022
2,SA-3905,185384799,Deporte,2022-12-12T15:34:28.000+0000,Phone,0018c00002ay6KFAAY,08q8c000000VW24AAG,12,2022,12,2022-12-12 15:34:28+00:00,12-12-2022
3,SA-3871,187821444,Psicología,2022-12-12T12:21:02.000+0000,Phone,0018c00002bI5vJAAS,08q8c000000VVsBAAW,12,2022,12,2022-12-12 12:21:02+00:00,12-12-2022
4,SA-3898,167466923,Nutrición,2022-12-12T15:04:05.000+0000,Phone,0018c00002ao4fCAAQ,08q8c000000VVs1AAG,12,2022,12,2022-12-12 15:04:05+00:00,12-12-2022


In [227]:
Gran_Citas_deservicio['Fecha'] = Gran_Citas_deservicio['Fecha'].dt.tz_localize(None)
Gran_Citas_deservicio.to_excel(
    "C:/Users/jmartinez/Documents/servicio.xlsx", index=False
)

In [238]:
Clinicas = Clinicas.rename(
    columns={
        
        "Fecha": "CreatedDate"
        
    }
)

Clinicas["Mes"] = pd.to_datetime(
    Clinicas["CreatedDate"]
).dt.month
Clinicas["Año"] = pd.to_datetime(
    Clinicas["CreatedDate"]
).dt.year
Clinicas["Dia"] = pd.to_datetime(
    Clinicas["CreatedDate"]
).dt.day

# x = date(Citas_de_servicio['Dia'], Citas_de_servicio['Mes'], Citas_de_servicio['Año'] )
Clinicas["Fecha"] = pd.to_datetime(
    Clinicas["CreatedDate"], dayfirst=True, yearfirst=False, exact=True
)
Clinicas["Fecha3"] = Clinicas["Fecha"].dt.strftime("%d-%m-%Y")


Clinicas['Fecha'] = Clinicas['Fecha'].dt.tz_localize(None)
Clinicas.to_excel(
    "C:/Users/jmartinez/Documents/clinicas.xlsx", index=False
)
Clinicas.dtypes



Codigo Ficha                    object
Rut                             object
Empleador                       object
CreatedDate                     object
Canal                           object
Cod. Empresa Emp_pro            object
Id Paciente                     object
Tipo Ficha                      object
clínica                         object
Mes                              int64
Año                              int64
Dia                              int64
Fecha                   datetime64[ns]
Fecha3                          object
dtype: object